In [45]:
# load pytorch library.

from torchtext.datasets import YelpReviewPolarity
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim


### Yelp Review Polarity Data Description
The Yelp reviews polarity dataset is constructed by considering stars 1 and 2 negative, and 3 and 4 positive. For each polarity 280,000 training samples and 19,000 testing samples are take randomly. In total there are 560,000 trainig samples and 38,000 testing samples. Negative polarity is class 1, and positive class 2.


In [49]:
# Load Yelp Reivew Polarity dataset.
train_iter, test_iter = YelpReviewPolarity(split=('train','test'))


In [53]:
# list a sample data.
next(train_iter)

(1,
 "I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you call the office, they'll put you through to a voice mail, that NO ONE ever answers or returns your call. Both my adult children and husband have decided to leave this practice after experiencing such frustration. The entire office has an attitude like they are doing you a favor. Give me a break! Stay away from this doc and the practice. You deserve better and they will not be there when you reall

In [51]:
next(test_iter)

(2,
 "Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \\nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. But here, my service and road coverage has all been well explained - and let up to me to decide. \\nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.")

In [31]:
dataloader = DataLoader(train_iter, batch_size=64, shuffle=False)

In [32]:
# initialize tokenizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [33]:
# Initialize english tokernizer
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

# build vocab object.
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])


In [34]:
# Build text and label pipeline
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [35]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)
    

In [38]:
train_iter = YelpReviewPolarity(split='train')
# data loader using collate_batch fn
dataloader = DataLoader(train_iter, batch_size=64, shuffle=False, collate_fn=collate_batch)

In [39]:
from torch import nn
# Text Classification Network
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)


In [40]:
# Define Network parameters
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [41]:
import time
# Model train
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) 
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

# Model validation
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count


In [42]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 5 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = YelpReviewPolarity()

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 8313 batches | accuracy    0.779
| epoch   1 |  1000/ 8313 batches | accuracy    0.863
| epoch   1 |  1500/ 8313 batches | accuracy    0.880
| epoch   1 |  2000/ 8313 batches | accuracy    0.889
| epoch   1 |  2500/ 8313 batches | accuracy    0.896
| epoch   1 |  3000/ 8313 batches | accuracy    0.898
| epoch   1 |  3500/ 8313 batches | accuracy    0.903
| epoch   1 |  4000/ 8313 batches | accuracy    0.900
| epoch   1 |  4500/ 8313 batches | accuracy    0.904
| epoch   1 |  5000/ 8313 batches | accuracy    0.905
| epoch   1 |  5500/ 8313 batches | accuracy    0.907
| epoch   1 |  6000/ 8313 batches | accuracy    0.911
| epoch   1 |  6500/ 8313 batches | accuracy    0.908
| epoch   1 |  7000/ 8313 batches | accuracy    0.907
| epoch   1 |  7500/ 8313 batches | accuracy    0.911
| epoch   1 |  8000/ 8313 batches | accuracy    0.911
-----------------------------------------------------------
| end of epoch   1 | time: 61.64s | valid accuracy    0.912 
---------------

### Result:
Pick any 2 datasets (except AG_NEWS) from torchtext.datasets and train your model on them achieving 50% more accuracy than random prediction. 
Upload to Github with a proper readme file describing your datasets, and showing your logs as well. 

1. Yelp Review Plarity data set was input data set for classification and this was two class classification problem.
2. Objective was to chieving 50% more accuracy(75%) than random prediction(50%) which was 75%
3. Highest validation accuracy acheived was 92.9% which was 50% more accuracy than random prediction

